In [1]:
%load_ext autoreload
%autoreload 2
# stockdata 불러오기 
from backtest.request.stockdata_from_repo import build_stock_data_from_repo_request
from backtest.use_cases.stockdata_from_repo import stockdata_from_repo
from backtest.repository.webrepo.upbit_repo import UpbitRepo
from backtest.repository.webrepo.bithumb_repo import BithumbRepo
from backtest.repository.webrepo.binance_repo import BinanceRepo

filter = {'order__eq': 'BTC', 'from__eq': '2017-12-01'}
request=build_stock_data_from_repo_request(filters=filter)
stockdata_bithumb = stockdata_from_repo(BithumbRepo(),request)
stockdata_upbit = stockdata_from_repo(UpbitRepo(),request)
stockdata_binance = stockdata_from_repo(BinanceRepo(),request)



2020-05-12
2017-08-17


In [2]:
stockdata_bithumb.value

StockData(symbol='', data=                           open        high         low       close  \
date                                                                  
2017-12-01 00:00:00  10899000.0  12588000.0  10442000.0  12233000.0   
2017-12-02 00:00:00  12220000.0  13091000.0  12030000.0  12663000.0   
2017-12-03 00:00:00  12663000.0  13375000.0  12515000.0  13259000.0   
2017-12-04 00:00:00  13259000.0  13490000.0  12791000.0  13401000.0   
2017-12-05 00:00:00  13404000.0  14385000.0  13311000.0  14237000.0   
...                         ...         ...         ...         ...   
2023-02-01 00:00:00  29054000.0  29237000.0  28638000.0  28991000.0   
2023-02-02 00:00:00  28993000.0  30228000.0  28651000.0  29626000.0   
2023-02-03 00:00:00  29628000.0  30100000.0  29250000.0  29506000.0   
2023-02-04 00:00:00  29505000.0  29736000.0  29262000.0  29550000.0   
2023-02-05 14:00:00  29551000.0  29572000.0  29350000.0  29440000.0   

                           volume  
date          

In [3]:
stockdata_binance.value

StockData(symbol='', data=                         open      high       low     close         volume
date                                                                      
2017-12-02 09:00:00  10775.04  11190.00  10620.00  10890.01    4765.439757
2017-12-03 09:00:00  10902.69  11825.00  10500.00  11165.41    5346.636524
2017-12-04 09:00:00  11165.41  11600.00  10802.00  11579.00    4663.424562
2017-12-05 09:00:00  11571.03  11853.00  11447.68  11699.99    5550.732055
2017-12-06 09:00:00  11699.99  13615.23  11665.58  13550.05    6707.946319
...                       ...       ...       ...       ...            ...
2023-02-01 09:00:00  23125.13  23812.66  22760.23  23732.66  310790.422710
2023-02-02 09:00:00  23731.41  24255.00  23363.27  23488.94  364177.207510
2023-02-03 09:00:00  23489.33  23715.70  23204.62  23431.90  332571.029040
2023-02-04 09:00:00  23431.90  23587.78  23253.96  23326.84  166126.472950
2023-02-05 09:00:00  23327.66  23390.00  23230.00  23371.89   34702.823180

In [4]:
stockdata_upbit.value

StockData(symbol='', data=                           open        high         low       close  \
date                                                                  
2017-12-01 09:00:00  11935000.0  12680000.0  11201000.0  12532000.0   
2017-12-02 09:00:00  12532000.0  13180000.0  12350000.0  12971000.0   
2017-12-03 09:00:00  12970000.0  13529000.0  12840000.0  13128000.0   
2017-12-04 09:00:00  13120000.0  13688000.0  13119000.0  13684000.0   
2017-12-05 09:00:00  13683000.0  14794000.0  13612000.0  14645000.0   
...                         ...         ...         ...         ...   
2023-02-01 09:00:00  29052000.0  29740000.0  28626000.0  29620000.0   
2023-02-02 09:00:00  29599000.0  30285000.0  29530000.0  29600000.0   
2023-02-03 09:00:00  29601000.0  29734000.0  29287000.0  29432000.0   
2023-02-04 09:00:00  29432000.0  29650000.0  29349000.0  29420000.0   
2023-02-05 09:00:00  29398000.0  29528000.0  29350000.0  29450000.0   

                           volume  
date          

In [5]:
# strategy function 제작
import pandas as pd
from backtest.domains.strategy_result import StrategyResult, StrategyResultColumnType
from backtest.domains.strategy import Strategy

def strategy_function1(strategy: Strategy):
    response = StrategyResult(value=pd.DataFrame(
        index=strategy.data.index, columns=[strategy.name]))
    response.value[strategy.name] = [(
        StrategyResultColumnType.SELL, strategy.weight)] * len(strategy)
    return response

In [6]:
from backtest.use_cases.strategy_execute import strategy_execute
from backtest.use_cases.strategy_execute import _basic_function
from backtest.domains.strategy import Strategy,StrategyType

strategy1 = Strategy(name='basic_strategy',type=StrategyType.with_stockdata,data=stockdata_upbit.value,weight=100)
strategy2 = Strategy(name='basic_strategy2',type=StrategyType.with_stockdata,data=stockdata_upbit.value,weight=300)

strategy_result1 = strategy_execute(strategy1)
strategy_result2 = strategy_execute(strategy2,strategy_fucntion=strategy_function1)

In [7]:
strategy_result1.value
# StrategyResultColumnType 종류
# KEEP, SELL, BUY

StrategyResult(value=                                           basic_strategy
date                                                     
2017-12-01 09:00:00  (StrategyResultColumnType.KEEP, 100)
2017-12-02 09:00:00  (StrategyResultColumnType.KEEP, 100)
2017-12-03 09:00:00  (StrategyResultColumnType.KEEP, 100)
2017-12-04 09:00:00  (StrategyResultColumnType.KEEP, 100)
2017-12-05 09:00:00  (StrategyResultColumnType.KEEP, 100)
...                                                   ...
2023-02-01 09:00:00  (StrategyResultColumnType.KEEP, 100)
2023-02-02 09:00:00  (StrategyResultColumnType.KEEP, 100)
2023-02-03 09:00:00  (StrategyResultColumnType.KEEP, 100)
2023-02-04 09:00:00  (StrategyResultColumnType.KEEP, 100)
2023-02-05 09:00:00  (StrategyResultColumnType.KEEP, 100)

[1893 rows x 1 columns])

In [8]:
strategy_result2.value

StrategyResult(value=                                          basic_strategy2
date                                                     
2017-12-01 09:00:00  (StrategyResultColumnType.SELL, 300)
2017-12-02 09:00:00  (StrategyResultColumnType.SELL, 300)
2017-12-03 09:00:00  (StrategyResultColumnType.SELL, 300)
2017-12-04 09:00:00  (StrategyResultColumnType.SELL, 300)
2017-12-05 09:00:00  (StrategyResultColumnType.SELL, 300)
...                                                   ...
2023-02-01 09:00:00  (StrategyResultColumnType.SELL, 300)
2023-02-02 09:00:00  (StrategyResultColumnType.SELL, 300)
2023-02-03 09:00:00  (StrategyResultColumnType.SELL, 300)
2023-02-04 09:00:00  (StrategyResultColumnType.SELL, 300)
2023-02-05 09:00:00  (StrategyResultColumnType.SELL, 300)

[1893 rows x 1 columns])